In [1]:
!pip install Pytrends
!pip install pandas
!pip install gspread-dataframe==3.2.1

  Attempting uninstall: gspread-dataframe
    Found existing installation: gspread-dataframe 4.0.0
    Uninstalling gspread-dataframe-4.0.0:
      Successfully uninstalled gspread-dataframe-4.0.0


In [2]:
!pip install gspread --upgrade # Update gspread to the latest version
!pip install --upgrade gspread-dataframe # Update gspread-dataframe to the latest version

  Attempting uninstall: gspread-dataframe
    Found existing installation: gspread-dataframe 3.2.1
    Uninstalling gspread-dataframe-3.2.1:
      Successfully uninstalled gspread-dataframe-3.2.1


In [6]:
# inflacion queries 2011-01-01 | 2024-12-31
SPREADSHEET = 'https://docs.google.com/spreadsheets/d/14BlhOuAZzdh7MdwbcB_H6jvM2A2b61a3AgkbH7vx6tg/edit#gid=0'

# inflacion weekly queries 2019-01-01 | 2023-12-31
#SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1r-tNR_D8GXXUAoIx6do1o1xt-0XwQYqX1dlNOUe-Azc/edit#gid=0'

# inflacion weekly queries 2019-03-01 | 2024-02-29
#SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1AxWHmrpKVdQA_DMiNup4UV_F9SDFrr8sCUm3P__iSKM/edit#gid=0'

# inflacion weekly queries 2023-01-01 | 2024-02-29
#SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1o_pOWewXcAhId5bCGpe_O2FDl7mN2WxPZkppRxCEjyk/edit?gid=0#gid=0'


import pandas as pd
from pytrends.request import TrendReq
from google.colab import auth
import gspread
from google.auth import default
import time
import matplotlib.pyplot as plt
import seaborn as sns
#from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Import Worksheet instead of Cell
from gspread import Worksheet, utils # Import Worksheet and utils from gspread

import urllib.parse
import pathlib

import contextlib

import requests
from pytrends.request import TrendReq

class _TrendReq(TrendReq):
    def GetGoogleCookie(self):
        # TODO: make sure to get rid of this dirty hack
        with _requests_get_as_post():
            return super().GetGoogleCookie()


@contextlib.contextmanager
def _requests_get_as_post():
    requests.get, requests_get = requests.post, requests.get
    try:
        yield
    finally:
        requests.get = requests_get


auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
startTime = time.time()
pytrend = _TrendReq(hl='en-GB', tz=360)
#pytrend = TrendReq()

df = wb = gc.open_by_url(SPREADSHEET)
sheet = wb.worksheet("Criteria")
df2 = sheet.col_values(5)
d_from = sheet.acell('B7').value
d_to = sheet.acell('B8').value


dataset_time = []
DELAY_TIME = 2
df_list = []


for geo1 in sheet.col_values(7):
  for x in range(0,len(df2)):

    keywords = [df2[x]]

    pytrend.build_payload(kw_list=keywords, cat=0, timeframe= str(d_from + " " + d_to), geo= str(geo1))

    done = False
    while not done:
      try:
        data_time = pytrend.interest_over_time()
        done = True
        break
      except:
        time.sleep(DELAY_TIME) # delay because too many requests

    if not data_time.empty:
      data_time['country']=geo1
      data_time['query'] = keywords[0]
      data_time['value'] = data_time[keywords[0]]
      data_time = data_time.drop(labels=['isPartial', keywords[0]],axis='columns')
      dataset_time.append(data_time)

    datalist = []
    for i in dataset_time:
      result = pd.DataFrame(i)
      datalist.append(result)


    for i in datalist:
      df_list.append(i)

new_df_list = []

for i in df_list:
  new_i = i.dropna(axis=1, how='all')
  new_df_list.append(new_i)

final_df_list = None
for i in new_df_list:
  if final_df_list is None:
    final_df_list = [i]
  else:
    hasdf = False
    for j in final_df_list:
      if i.equals(j):
        hasdf = True
    if not hasdf:
      final_df_list.append(i)

new_final = []
for i in final_df_list:
  new_i = i.reset_index().set_index(['date'])
  new_final.append(new_i)

big_df = pd.concat(new_final).reset_index()
big_df['date'] = pd.to_datetime(big_df['date']).dt.date
big_df = big_df.set_index(['date'])

wsResults = wb.worksheet("Results")
wsResults.clear()
# Use set_with_dataframe from gspread_dataframe to write the DataFrame
from gspread_dataframe import set_with_dataframe # This line should remain unchanged
set_with_dataframe(wsResults, big_df,include_index=True)